In [1]:
from qtnmtts.circuits.lcu import LCUMultiplexorBox
import numpy as np


### Example 1 - Three-qubit Ising model Hamiltonian

In [2]:
from qtnmtts.operators import ising_model

n_state_qubits = 3
h = 1
j = 1
ising_model_3q = ising_model(3, h, j)
print(ising_model_3q)


{(Zq[0], Zq[1]): 1.00000000000000, (Zq[1], Zq[2]): 1.00000000000000, (Xq[0]): 1.00000000000000, (Xq[1]): 1.00000000000000, (Xq[2]): 1.00000000000000}


Success probability is inversely related to the square of the l1-norm of the operator.
It also depends on the initial state of the system. 
We construct a random normalized initial statevector and use `StatePreparationBox` to prepare this.

In [3]:
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import StatePreparationBox 
from qtnmtts.circuits.core import QRegMap

np.random.seed(10)
state0 = np.random.rand(2**n_state_qubits) + (
        1j * np.random.rand(2**n_state_qubits)
    )
state0 = state0 / np.sqrt(state0.conj().T @ state0)
state0_cbox = StatePreparationBox(state0)

lcu_box = LCUMultiplexorBox(ising_model_3q,n_state_qubits)
lcu_box_circ = lcu_box.get_circuit()
lcu_circ = lcu_box.initialise_circuit()
lcu_circ.add_gate(state0_cbox,lcu_box.qreg.state) #add initial state
qreg_map = QRegMap(lcu_box.q_registers, lcu_circ.q_registers)
lcu_circ.add_registerbox(lcu_box, qreg_map)
render_circuit_jupyter(lcu_circ)

We then do shot measurements and count the number of shots which yield states in the subspace of the Hamiltonian above. (Upper left block of the matrix.)

In [4]:
cp = lcu_circ.add_c_register("cp", lcu_box.qreg.prepare.size) #len(lcu_box._p)
cq = lcu_circ.add_c_register('cq', lcu_box.qreg.state.size)

for ii in range(lcu_box.qreg.prepare.size):
    lcu_circ.Measure(lcu_box.qreg.prepare[ii],cp[ii])
for ii in range(lcu_box.qreg.state.size):
    lcu_circ.Measure(lcu_box.qreg.state[ii],cq[ii])

from pytket.extensions.qulacs import QulacsBackend
backend = QulacsBackend()
n_shots = 2**12
compiled_circ = backend.get_compiled_circuit(lcu_circ, optimisation_level=1)
print('running...')

result = backend.run_circuit(compiled_circ, n_shots=n_shots)
result._state = None
result_dist = result.get_counts() #get_distribution()
p_reg_size = lcu_box.qreg.prepare.size
lcu_post_select_dist =  {str(kk[p_reg_size:]) : v for kk,v in result_dist.items() if kk[:p_reg_size] == tuple(0 for i in range(p_reg_size))}
print(f'total shots: {n_shots}, successful shots: {sum(lcu_post_select_dist.values())}')
print(f'final state distribution: {lcu_post_select_dist}')
print(f'measurement success probability: {sum(lcu_post_select_dist.values())/n_shots}')

running...
total shots: 4096, successful shots: 1549
final state distribution: {'(0, 0, 0)': 279, '(0, 0, 1)': 202, '(0, 1, 0)': 10, '(0, 1, 1)': 115, '(1, 0, 0)': 118, '(1, 0, 1)': 24, '(1, 1, 0)': 191, '(1, 1, 1)': 610}
measurement success probability: 0.378173828125


### Analytical success probability and amplified success probability

In [5]:
lcu_block_mat = ising_model_3q.to_sparse_matrix().todense()
statef = lcu_block_mat @ state0
alpha = lcu_box.l1_norm
success_probability = (statef @ statef.conj().transpose())/np.abs(alpha)**2
print(f'success probability (analytical): {success_probability}')

statef_amp = ((3 * lcu_block_mat) - ( (4/alpha**2) * lcu_block_mat @ lcu_block_mat.conj().transpose() @ lcu_block_mat)) @ state0
success_probability_amp = ( statef_amp @ statef_amp.conj().transpose())/alpha**2
print(f'amplified success probability (analytical): {success_probability_amp}')

success probability (analytical): [[0.36954907+0.j]]
amplified success probability (analytical): [[0.49388257+0.j]]


Let's see if including an `AmplificationBox` will result in the expected amplification above.

In [6]:
from qtnmtts.circuits.amplitude_amplification import AmplificationBox
n_iterations = 1
amp_box = AmplificationBox(lcu_box, n_iterations)
print('Inside the AmplificationBox:')
render_circuit_jupyter(amp_box.reg_circuit)



Inside the AmplificationBox:


In [7]:
lcu_amp_circ = lcu_box.initialise_circuit()
lcu_amp_circ.add_gate(state0_cbox,lcu_box.qreg.state) #add initial state
# circ.q_registers
qreg_map = QRegMap(lcu_box.q_registers, lcu_amp_circ.q_registers)
lcu_amp_circ.add_registerbox(lcu_box, qreg_map)
qreg_map2 = QRegMap(amp_box.q_registers, lcu_amp_circ.q_registers)
lcu_amp_circ.add_registerbox(amp_box, qreg_map2)
print('Amplified LCU circuit:')
render_circuit_jupyter(lcu_amp_circ)


Amplified LCU circuit:


Run and measure the amplified circuit for n_shots...

In [8]:
cp = lcu_amp_circ.add_c_register("cp", lcu_box.qreg.prepare.size) #len(lcu_box._p)
cq = lcu_amp_circ.add_c_register('cq', lcu_box.qreg.state.size)

for ii in range(lcu_box.qreg.prepare.size):
    lcu_amp_circ.Measure(lcu_box.qreg.prepare[ii],cp[ii])
for ii in range(lcu_box.qreg.state.size):
    lcu_amp_circ.Measure(lcu_box.qreg.state[ii],cq[ii])
compiled_circ = backend.get_compiled_circuit(lcu_amp_circ, optimisation_level=1)
print('running...')

result = backend.run_circuit(compiled_circ, n_shots=n_shots)
result._state = None
result_dist = result.get_counts() #get_distribution()
p_reg_size = lcu_box.qreg.prepare.size
amp_lcu_post_select_dist =  {str(kk[p_reg_size:]) : v for kk,v in result_dist.items() if kk[:p_reg_size] == tuple(0 for i in range(p_reg_size))}


print(f'total shots: {n_shots}, successful shots: {sum(amp_lcu_post_select_dist.values())}')
print(f'final state distribution: {amp_lcu_post_select_dist}')
print(f'measurement success probability: {sum(amp_lcu_post_select_dist.values())/n_shots}, analytical est.: {success_probability_amp}')
print(f'Without amplification, the measurement success probability was {sum(lcu_post_select_dist.values())/n_shots} (analytical est.: {success_probability})')

running...
total shots: 4096, successful shots: 2025
final state distribution: {'(0, 0, 0)': 242, '(0, 0, 1)': 307, '(0, 1, 0)': 1, '(0, 1, 1)': 143, '(1, 0, 0)': 111, '(1, 0, 1)': 28, '(1, 1, 0)': 283, '(1, 1, 1)': 910}
measurement success probability: 0.494384765625, analytical est.: [[0.49388257+0.j]]
Without amplification, the measurement success probability was 0.378173828125 (analytical est.: [[0.36954907+0.j]])


### Example 2 -- 

In [9]:
from pytket.utils.operators import QubitPauliOperator
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit

hamiltonian = QubitPauliOperator(
        {
            QubitPauliString(
                [Qubit(0), Qubit(1), Qubit(2)], [Pauli.Z, Pauli.X, Pauli.Y]
            ): -0.5,
            QubitPauliString(
                [Qubit(0), Qubit(1), Qubit(2)], [Pauli.X, Pauli.Z, Pauli.Z]
            ): -0.1,
            QubitPauliString(
                [Qubit(0), Qubit(1), Qubit(2)], [Pauli.Y, Pauli.Y, Pauli.X]
            ): -0.2j,
            QubitPauliString(
                [Qubit(0), Qubit(1), Qubit(2)], [Pauli.X, Pauli.X, Pauli.Y]
            ): -0.3,
        })

In [10]:
n_state_qubits = len(hamiltonian.all_qubits)
np.random.seed(10)
state0 = np.random.rand(2**n_state_qubits) + (
        1j * np.random.rand(2**n_state_qubits)
    )
state0 = state0 / np.sqrt(state0.conj().T @ state0)
state0_cbox = StatePreparationBox(state0)

lcu_box = LCUMultiplexorBox(hamiltonian,n_state_qubits)
lcu_box_circ = lcu_box.get_circuit()
lcu_circ = lcu_box.initialise_circuit()
lcu_circ.add_gate(state0_cbox,lcu_box.qreg.state) #add initial state
qreg_map = QRegMap(lcu_box.q_registers, lcu_circ.q_registers)
lcu_circ.add_registerbox(lcu_box, qreg_map)

cp = lcu_circ.add_c_register("cp", lcu_box.qreg.prepare.size) #len(lcu_box._p)
cq = lcu_circ.add_c_register('cq', lcu_box.qreg.state.size)

for ii in range(lcu_box.qreg.prepare.size):
    lcu_circ.Measure(lcu_box.qreg.prepare[ii],cp[ii])
for ii in range(lcu_box.qreg.state.size):
    lcu_circ.Measure(lcu_box.qreg.state[ii],cq[ii])


n_shots = 2**12
compiled_circ = backend.get_compiled_circuit(lcu_circ, optimisation_level=1)
print('running LCU...')

result = backend.run_circuit(compiled_circ, n_shots=n_shots)
result._state = None
result_dist = result.get_counts() #get_distribution()
p_reg_size = lcu_box.qreg.prepare.size
lcu_post_select_dist =  {str(kk[p_reg_size:]) : v for kk,v in result_dist.items() if kk[:p_reg_size] == tuple(0 for i in range(p_reg_size))}
print(f'total shots: {n_shots}, successful shots: {sum(lcu_post_select_dist.values())}')
print(f'final state distribution: {lcu_post_select_dist}')
print(f'measurement success probability: {sum(lcu_post_select_dist.values())/n_shots}')

lcu_block_mat = hamiltonian.to_sparse_matrix().todense()
statef = lcu_block_mat @ state0
alpha = lcu_box.l1_norm
success_probability = (statef @ statef.conj().transpose())/np.abs(alpha)**2
print(f'success probability (analytical): {success_probability}')

statef_amp = ((3 * lcu_block_mat) - ( (4/alpha**2) * lcu_block_mat @ lcu_block_mat.conj().transpose() @ lcu_block_mat)) @ state0
success_probability_amp = ( statef_amp @ statef_amp.conj().transpose())/alpha**2
print(f'amplified success probability (analytical): {success_probability_amp}')

n_iterations = 1
amp_box = AmplificationBox(lcu_box, n_iterations)
lcu_amp_circ = lcu_box.initialise_circuit()
lcu_amp_circ.add_gate(state0_cbox,lcu_box.qreg.state.to_list()) #add initial state
qreg_map = QRegMap(lcu_box.q_registers, lcu_amp_circ.q_registers)
lcu_amp_circ.add_registerbox(lcu_box, qreg_map)
qreg_map2 = QRegMap(amp_box.q_registers, lcu_amp_circ.q_registers)
lcu_amp_circ.add_registerbox(amp_box, qreg_map2)

cp = lcu_amp_circ.add_c_register("cp", lcu_box.qreg.prepare.size) #len(lcu_box._p)
cq = lcu_amp_circ.add_c_register('cq', lcu_box.qreg.state.size)

for ii in range(lcu_box.qreg.prepare.size):
    lcu_amp_circ.Measure(lcu_box.qreg.prepare[ii],cp[ii])
for ii in range(lcu_box.qreg.state.size):
    lcu_amp_circ.Measure(lcu_box.qreg.state[ii],cq[ii])
compiled_circ = backend.get_compiled_circuit(lcu_amp_circ, optimisation_level=1)
print('running with AmplificationBox...')

result = backend.run_circuit(compiled_circ, n_shots=n_shots)
result._state = None
result_dist = result.get_counts() #get_distribution()
p_reg_size = lcu_box.qreg.prepare.size
amp_lcu_post_select_dist =  {str(kk[p_reg_size:]) : v for kk,v in result_dist.items() if kk[:p_reg_size] == tuple(0 for i in range(p_reg_size))}


print(f'total shots: {n_shots}, successful shots: {sum(amp_lcu_post_select_dist.values())}')
print(f'final state distribution: {amp_lcu_post_select_dist}')
print(f'measurement success probability: {sum(amp_lcu_post_select_dist.values())/n_shots}, analytical est.: {success_probability_amp}')
print(f'Without amplification, the measurement success probability was {sum(lcu_post_select_dist.values())/n_shots} (analytical est.: {success_probability})')


running LCU...
total shots: 4096, successful shots: 1489
final state distribution: {'(0, 0, 0)': 806, '(0, 0, 1)': 181, '(0, 1, 0)': 14, '(0, 1, 1)': 322, '(1, 0, 0)': 79, '(1, 0, 1)': 37, '(1, 1, 0)': 44, '(1, 1, 1)': 6}
measurement success probability: 0.363525390625
success probability (analytical): [[0.36682009+3.58413941e-18j]]
amplified success probability (analytical): [[0.54883225+0.j]]
running with AmplificationBox...
total shots: 4096, successful shots: 2218
final state distribution: {'(0, 0, 0)': 806, '(0, 0, 1)': 714, '(0, 1, 0)': 120, '(0, 1, 1)': 257, '(1, 0, 0)': 85, '(1, 0, 1)': 93, '(1, 1, 0)': 77, '(1, 1, 1)': 66}
measurement success probability: 0.54150390625, analytical est.: [[0.54883225+0.j]]
Without amplification, the measurement success probability was 0.363525390625 (analytical est.: [[0.36682009+3.58413941e-18j]])
